In [ ]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [4]:
import tensorflow as tf
import numpy as np
import copy
import tqdm
import IProgress
from hfunc import models
from hfunc import metrics

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
class_accuracy = metrics.ClassAccuracy()

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=200)

In [ ]:
model.evaluate(x_test, y_test, verbose=2)

In [ ]:
l, a = model.evaluate(x_test, y_test, verbose=2)
or_weights = model.get_weights()
size = 128
for i in range(128):
    w = copy.deepcopy(or_weights)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model.set_weights(w)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    print(f"Node {i}:", 0.*(na - a) + 1.0*(l - nl))

In [ ]:
loss, acc = model.evaluate(x_test, y_test, verbose=2)
end_not_reached = True
improved = False
size = 128
tol = -1e-30
current_pos = 0
best_pos = -1
best_change = tol
original2 = model.get_weights()
bas2 = [acc]
bls2 = [loss]
best_weights2 = model.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
while end_not_reached or improved:
    if not(end_not_reached):
        end_not_reached = True
        improved = False
        current_pos = 0
        size -= 1
        nodes_removed2 += [best_pos]
        best_weights2[0][:,best_pos] = 0
        best_weights2[1][best_pos] = 0
        best_weights2[2][best_pos,:] = 0
        best_pos = -1
        #tol -= best_change
        ol = best_loss
        oa = best_acc
        bas2 += [best_acc]
        bls2 += [best_loss]
        print("Improvement has occured!! Accuracy:", best_acc, "--- Loss:", best_loss, '--- Change:', best_change, '--- New tol:', tol)
        best_change = tol
        num_removed2 += 1
    if current_pos in nodes_removed2:
        current_pos += 1
        if current_pos - num_removed2 >= size:
            end_not_reached = False
        continue
    w = copy.deepcopy(best_weights2)
    w[0][:,current_pos] = 0
    w[1][current_pos] = 0
    w[2][current_pos,:] = 0
    tester_model.set_weights(w)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    if 0.1*(na - oa) + 0.9*(ol - nl) > best_change:
        best_change = 0.1*(na - oa) + 0.9*(ol - nl)
        print(best_change)
        best_pos = current_pos
        improved = True
        best_acc = na
        best_loss = nl
        print("Found something better")
    current_pos += 1
    if current_pos - num_removed2 >= size:
        end_not_reached = False
    if current_pos%20 == 0:
        print("Did 20 iterations")

tester_model.set_weights(best_weights2)
loss2, acc2 = tester_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
for i in nodes_removed2:
    best_weights2[0][:,i] = np.random.normal(0, 2/np.sqrt(28*28 + 128), 784)
    best_weights2[1][i] = 0
    best_weights2[2][i,:] = np.random.normal(0, 2/np.sqrt(138), 10)

In [ ]:
new_model = tf.keras.models.Sequential()
new_model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
new_model.add(tf.keras.layers.Dense(128, activation='relu'))
new_model.add(tf.keras.layers.Dense(10, activation='softmax'))

new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

new_model.set_weights(best_weights2)

new_model.evaluate(x_test, y_test, verbose=2)
new_model.fit(x_train, y_train, epochs=5)
new_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
loss, acc = model.evaluate(x_test, y_test, verbose=2)
end_not_reached = True
improved = False
size = 128
tol = -1e-30
current_pos = 0
best_pos = -1
best_change = tol
original2 = model.get_weights()
bas2 = [acc]
bls2 = [loss]
best_weights2 = model.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
l = loss
a = acc
num_removed2 = 0

for i in range(128):
    w = copy.deepcopy(original2)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model.set_weights(w)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    change = 0.*(na - a) + 1.0*(l - nl)
    print(f"Node {i}:", change)
    if change > tol:
        nodes_removed2 += [i]
        num_removed2 += 1
        
for i in nodes_removed2:
    best_weights2[0][:,i] = 0
    best_weights2[1][i] = 0
    best_weights2[2][i,:] = 0

tester_model.set_weights(best_weights2)
loss2, acc2 = tester_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
l, a = model.evaluate(x_test, y_test, verbose=2)
or_weights = model.get_weights()
size = 128
worst_remove = -1
wc = 0
w2 = model.get_weights()
for i in range(128):
    w = copy.deepcopy(or_weights)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model.set_weights(w)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    print(f"Node {i}:", 0.*(na - a) + 1.0*(l - nl))
    if 0.*(na - a) + 1.0*(l - nl) < wc:
        worst_remove = i
        wc = (l - nl)
w2[0][:,worst_remove] = 0
w2[1][worst_remove] = 0
w2[2][worst_remove,:] = 0
tester_model.set_weights(w2)
loss2, acc2 = tester_model.evaluate(x_test, y_test, verbose=2)
l = loss2
a = acc2
for i in range(128):
    w = copy.deepcopy(w2)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model.set_weights(w)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    print(f"Node {i}:", 0.*(na - a) + 1.0*(l - nl))